In [ ]:
pip install torch>=1.8,<1.9

In [ ]:
pip install pyannote.audio

In [ ]:
pip install lightning-fabric


In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token="")


In [ ]:
pip install pydub


In [75]:
import time
import psutil

In [77]:
process = psutil.Process()
cpu_start = process.cpu_percent()
start = time.time()

In [78]:
import os
from pyannote.audio.pipelines.utils.hook import ProgressHook

with ProgressHook() as hook:


    diarization = pipeline("audio.wav")


output_dir = "diarization_output"
os.makedirs(output_dir, exist_ok=True)

rttm_path = os.path.join(output_dir, "audio.rttm")

with open(rttm_path, "w") as rttm_file:
    diarization.write_rttm(rttm_file)

Output()

In [93]:
end = time.time()
cpu_end = process.cpu_percent()
latency = end - start
cpu_usage = cpu_end - cpu_start


latency / 60 

26.566001427173614

In [83]:
import pandas as pd

data = pd.read_csv('./diarization_output/audio.rttm', delimiter=r'\s+', header=None)

data.columns = ["Type", "Source", "ID", "Start_Time", "Duration",  
                "Other1", "Other2", "Speaker", "Other3", "Other4"]

print(data.head())

      Type Source  ID  Start_Time  Duration  Other1  Other2     Speaker  \
0  SPEAKER  audio   1       1.381     2.177     NaN     NaN  SPEAKER_01   
1  SPEAKER  audio   1       4.790     1.603     NaN     NaN  SPEAKER_01   
2  SPEAKER  audio   1       7.287     0.675     NaN     NaN  SPEAKER_01   
3  SPEAKER  audio   1       9.127     3.578     NaN     NaN  SPEAKER_01   
4  SPEAKER  audio   1      12.873     2.059     NaN     NaN  SPEAKER_01   

   Other3  Other4  
0     NaN     NaN  
1     NaN     NaN  
2     NaN     NaN  
3     NaN     NaN  
4     NaN     NaN  


In [84]:
from collections import defaultdict

speaker_segments = defaultdict(list)  # Map<Speaker, List<Segment>>

def split_audio_by_speaker(audio, df, min_duration_sec=4):

    for index, row in df.iterrows():
        speaker = row['Speaker']
        start_ms = row['Start_Time'] * 1000  
        duration_ms = row['Duration'] * 1000  
        
        if duration_ms >= min_duration_sec * 1000:  
            segment = audio[start_ms:start_ms + duration_ms]
            
            speaker_segments[speaker].append(segment)

In [85]:
from pydub import AudioSegment

audio = AudioSegment.from_file("./audio.wav")

split_audio_by_speaker(audio, data)


In [86]:

for speaker, segments in speaker_segments.items():
    for i, segment in enumerate(segments):
        segment.export(os.path.join(output_dir, f"{speaker}_segment_{i+1}.wav"), format="wav")
        print(f"Exported {speaker}_segment_{i+1}.wav to {output_dir}")

print(f"Audio segments saved in the '{output_dir}' directory.")

Exported SPEAKER_01_segment_1.wav to diarization_output
Exported SPEAKER_01_segment_2.wav to diarization_output
Exported SPEAKER_01_segment_3.wav to diarization_output
Exported SPEAKER_01_segment_4.wav to diarization_output
Exported SPEAKER_01_segment_5.wav to diarization_output
Exported SPEAKER_01_segment_6.wav to diarization_output
Exported SPEAKER_01_segment_7.wav to diarization_output
Exported SPEAKER_01_segment_8.wav to diarization_output
Exported SPEAKER_01_segment_9.wav to diarization_output
Exported SPEAKER_01_segment_10.wav to diarization_output
Exported SPEAKER_01_segment_11.wav to diarization_output
Exported SPEAKER_01_segment_12.wav to diarization_output
Exported SPEAKER_01_segment_13.wav to diarization_output
Exported SPEAKER_01_segment_14.wav to diarization_output
Exported SPEAKER_01_segment_15.wav to diarization_output
Exported SPEAKER_01_segment_16.wav to diarization_output
Exported SPEAKER_01_segment_17.wav to diarization_output
Exported SPEAKER_01_segment_18.wav to di